# Standard Optimal Power Flow

In [122]:
import optalg
import pfnet

## Parse network

In [123]:
parser = pfnet.ParserMAT()
net = parser.parse('../../cases/case1354pegase.mat')
net.show_components()


Network Components
------------------
buses            : 1354
  slack          : 1
  reg by gen     : 260
  reg by tran    : 0
  reg by shunt   : 0
shunts           : 1082
  fixed          : 1082
  switched v     : 0
branches         : 1991
  lines          : 1751
  fixed trans    : 240
  phase shifters : 0
  tap changers v : 0
  tap changers Q : 0
generators       : 260
  slack          : 1
  reg            : 260
  P adjust       : 260
loads            : 673
  P adjust       : 0
vargens          : 0
batteries        : 0



## Construct problem

In [124]:
net.set_flags('bus',['variable','bounded'],'any','voltage magnitude')
net.set_flags('bus','variable','not slack','voltage angle')
net.set_flags('generator',['variable','bounded'],'any',['active power','reactive power'])

problem = pfnet.Problem()
problem.set_network(net)
problem.add_function(pfnet.Function('generation cost',1./net.num_generators,net))
problem.add_constraint(pfnet.Constraint('AC power balance',net))
problem.add_constraint(pfnet.Constraint('variable bounds',net))
problem.analyze()

## Solve problem

In [125]:
solver = optalg.opt_solver.OptSolverAugL()
solver.set_parameters({'quiet':True, 'feastol':1e-4,'optol':1e-4})
solver.solve(problem)
print(solver.get_status())

solved


## Get results

In [126]:
net.update_properties()
print(net.gen_P_cost, net.bus_P_mis)
net.set_var_values(solver.get_primal_variables())
net.update_properties()
print(net.gen_P_cost, net.bus_P_mis)

74752.9399999999 1299.7851457325244
73897.79971678853 0.0037510086070113857
